In [2]:
# note Grakel does not seem to support Python >=3.10, Python 3.9 works fine
# you are free to remove imports that are not useful for you
from grakel.datasets import fetch_dataset
from grakel.kernels import WeisfeilerLehman, VertexHistogram
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import KernelPCA # to check your own implementation
from sklearn.manifold import TSNE
import numpy as np
import scipy
import matplotlib.pyplot as plt
import math

In [3]:
# Some datasets, more datasets here https://ls11-www.cs.tu-dortmund.de/staff/morris/graphkerneldatasets

"""
    The MUTAG dataset consists of 188 chemical compounds divided into two 
    classes according to their mutagenic effect on a bacterium. 

    The chemical data was obtained form http://cdb.ics.uci.edu and converted 
    to graphs, where vertices represent atoms and edges represent chemical 
    bonds. Explicit hydrogen atoms have been removed and vertices are labeled
    by atom type and edges by bond type (single, double, triple or aromatic).
    Chemical data was processed using the Chemistry Development Kit (v1.4).
"""

"""
    ENZYMES is a dataset of protein tertiary structures obtained from (Borgwardt et al., 2005) 
    consisting of 600 enzymes from the BRENDA enzyme database (Schomburg et al., 2004). 
    In this case the task is to correctly assign each enzyme to one of the 6 EC top-level 
    classes. 
"""

"""
    NCI1 and NCI109 represent two balanced subsets of datasets of chemical compounds screened 
    for activity against non-small cell lung cancer and ovarian cancer cell lines respectively
    (Wale and Karypis (2006) and http://pubchem.ncbi.nlm.nih.gov).
"""

dataset = fetch_dataset("MUTAG", verbose=False) # just replace by the name of the datasets you want "ENZYMES", "NCI1"
G = dataset.data   # NB : this is a list of graphs, represented by dictionaries : first entry is an adjacency matrix, second are the node labels
y = dataset.target


dataset_MUT = fetch_dataset("MUTAG", verbose=False)
G_MUT = dataset_MUT.data
y_MUT = dataset_MUT.target

dataset_ENZ = fetch_dataset("ENZYMES", verbose=False)
G_ENZ = dataset_ENZ.data
y_ENZ = dataset_ENZ.target

dataset_NCI = fetch_dataset("NCI1", verbose=False)
G_NCI = dataset_NCI.data
y_NCI = dataset_NCI.target

In [12]:
# Computing the pairwise kernels between all graphs in the dataset, for each of the 3 datasets
gk = WeisfeilerLehman(n_iter=10, base_graph_kernel=VertexHistogram)   # n_iter=10 (parameter H) chosen arbitrarily, to coincide with question 4.1.4
K_MUT = gk.fit_transform(G_MUT)
K_ENZ = gk.fit_transform(G_ENZ)
K_NCI = gk.fit_transform(G_NCI)

print(K_NCI)

[[ 467  267  333 ...  455  397  424]
 [ 267  718  288 ...  755  671  652]
 [ 333  288  935 ...  482  429  446]
 ...
 [ 455  755  482 ... 1929 1231 1216]
 [ 397  671  429 ... 1231 1402 1020]
 [ 424  652  446 ... 1216 1020 1673]]


In [13]:
# Computing the rank of the WL subtree kernel matrix for all three datasets, and H=10 iterations
from numpy.linalg import matrix_rank
# NB : if n_iter has been changed from 10 in the previous cell, the kernel matrices need to be recomputed with n_iter=10 here !
print("MUTAG : kernel matrix rank : " + str(matrix_rank(K_MUT)) + ", number of samples : " + str(len(G_MUT)))
print("ENZYMES : kernel matrix rank : " + str(matrix_rank(K_ENZ)) + ", number of samples : " + str(len(G_ENZ)))
print("NCI1 : kernel matrix rank : " + str(matrix_rank(K_NCI)) + ", number of samples : " + str(len(G_NCI)))

MUTAG : kernel matrix rank : 175, number of samples : 188
ENZYMES : kernel matrix rank : 595, number of samples : 600
NCI1 : kernel matrix rank : 4002, number of samples : 4110
